<a href="https://colab.research.google.com/github/ChaitanyaTumu/starting_project/blob/main/BERT_trained_(good).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas transformers datasets torch openpyxl
!pip install transformers
!pip install torch
!pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
import pandas as pd

# Corrected Google Sheets CSV export link
file_path = "https://docs.google.com/spreadsheets/d/14_iFXrR7I5DXoZ3LOlwG8PKsT3CBs_VC/export?format=csv"
df = pd.read_csv(file_path)

# Load the sheet into a DataFrame
#print(df.head())
# Function to expand test cases into separate rows
# Function to expand test cases into separate rows
def expand_test_cases(row):
    test_cases = row['Test Case IDs'].split(', ')
    descriptions = row['Test Case Descriptions'].split('; ')
    if len(test_cases) != len(descriptions):
        raise ValueError(f"Mismatched lengths in row: {row}")
    return [
        {
            'Requirement ID': row['Requirement ID'],
            'Requirement Description': row['Requirement Description'],
            'Test Case ID': test_case,
            'Test Case Description': desc,
        }
        for test_case, desc in zip(test_cases, descriptions)
    ]

# Expand all rows
expanded_data = []
for _, row in df.iterrows():
    expanded_data.extend(expand_test_cases(row))

# Convert to DataFrame
df_expanded = pd.DataFrame(expanded_data)
print(df_expanded)


   Requirement ID                            Requirement Description  \
0          RQ9889  The system shall support automatic calibration...   
1          RQ9889  The system shall support automatic calibration...   
2          RQ9889  The system shall support automatic calibration...   
3          RQ8023  Each camera's yaw, pitch, and roll angles shal...   
4          RQ8023  Each camera's yaw, pitch, and roll angles shal...   
5          RQ8023  Each camera's yaw, pitch, and roll angles shal...   
6          RQ5047  Calibration data shall be transmitted over the...   
7          RQ5047  Calibration data shall be transmitted over the...   
8          RQ5047  Calibration data shall be transmitted over the...   
9          RQ4705  Calibration accuracy shall remain unaffected b...   
10         RQ4705  Calibration accuracy shall remain unaffected b...   
11         RQ4705  Calibration accuracy shall remain unaffected b...   
12         RQ1953  The system shall detect misaligned cameras an

In [5]:
import random

# Create negative pairs
negative_pairs = []

# Iterate over each requirement
for _, row in df_expanded.iterrows():
    requirement = row['Requirement Description']

    # Select a random test case from other rows as a negative pair
    negative_test_case = random.choice(df_expanded[df_expanded['Requirement ID'] != row['Requirement ID']]['Test Case Description'].values)

    # Add the negative pair
    negative_pairs.append({
        'Requirement Description': requirement,
        'Test Case Description': negative_test_case,
        'Label': 0
    })

# Combine positive and negative pairs
positive_pairs = [{'Requirement Description': row['Requirement Description'],
                   'Test Case Description': row['Test Case Description'],
                   'Label': 1} for _, row in df_expanded.iterrows()]

all_pairs = positive_pairs + negative_pairs
print(all_pairs)


[{'Requirement Description': 'The system shall support automatic calibration of all four cameras during initialization.', 'Test Case Description': 'Validate that all four cameras are automatically calibrated within 2 seconds after power-on.', 'Label': 1}, {'Requirement Description': 'The system shall support automatic calibration of all four cameras during initialization.', 'Test Case Description': 'Verify the system recalibrates successfully after a power cycle.', 'Label': 1}, {'Requirement Description': 'The system shall support automatic calibration of all four cameras during initialization.', 'Test Case Description': 'Simulate camera disconnection and verify system behavior during calibration.', 'Label': 1}, {'Requirement Description': "Each camera's yaw, pitch, and roll angles shall be adjustable within ±5 degrees of their specified values.", 'Test Case Description': 'Adjust yaw, pitch, and roll to ±5 degrees and verify system response.', 'Label': 1}, {'Requirement Description': "

In [7]:
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
import torch
from sklearn.model_selection import train_test_split


# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [12]:
tokenized_data = []

for pair in all_pairs:
    # Tokenize requirement and test case
    encoding = tokenizer.encode_plus(
        pair['Requirement Description'],
        pair['Test Case Description'],
        add_special_tokens=True,
        max_length=128,
        truncation=True,
        padding='max_length',
        return_tensors='pt'
    )

    tokenized_data.append({
        'requirement': pair['Requirement Description'],
        'test_case': pair['Test Case Description'],
        'input_ids': encoding['input_ids'].squeeze(),
        'attention_mask': encoding['attention_mask'].squeeze(),
        'label': pair['Label']
    })


print(tokenized_data)



[{'requirement': 'The system shall support automatic calibration of all four cameras during initialization.', 'test_case': 'Validate that all four cameras are automatically calibrated within 2 seconds after power-on.', 'input_ids': tensor([  101,  1996,  2291,  4618,  2490,  6882, 10250, 12322,  8156,  1997,
         2035,  2176,  8629,  2076,  3988,  3989,  1012,   102,  9398,  3686,
         2008,  2035,  2176,  8629,  2024,  8073, 10250, 12322,  9250,  2306,
         1016,  3823,  2044,  2373,  1011,  2006,  1012,   102,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,   

In [16]:
tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
print(tokens)

['[CLS]', 'the', 'system', 'shall', 'support', 'automatic', 'cal', '##ib', '##ration', 'of', 'all', 'four', 'cameras', 'during', 'initial', '##ization', '.', '[SEP]', 'valid', '##ate', 'that', 'all', 'four', 'cameras', 'are', 'automatically', 'cal', '##ib', '##rated', 'within', '2', 'seconds', 'after', 'power', '-', 'on', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD

In [70]:
for index, row in df_expanded.iterrows():
    requirement = row['Requirement Description']
    test_cases = row['Test Case Description']  # Assuming these are test case descriptions

    # Filter tokenized data for this requirement
    relevant_tokenized = [td for td in tokenized_data if td['requirement'] == requirement]


    for tokenized in relevant_tokenized:
        print(f"Requirement: {requirement}")
        print(f"Test Case: {tokenized['test_case']}")
        print(f"Tokenized Input IDs: {tokenized['input_ids']}")
        print(f"Tokenized Attention Mask: {tokenized['attention_mask']}")





Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
Requirement: Each camera's yaw, pitch, and roll angles shall be adjustable within ±5 degrees of their specified values.
Test Case: Verify the system's ability to return to default angles after recalibration.
Tokenized Input IDs: tensor([  101,  2169,  4950,  1005,  1055,  8038,  2860,  1010,  6510,  1010,
         1998,  4897, 12113,  4618,  2022, 26404,  2306,  1081,  2629,  5445,
         1997,  2037,  9675,  5300,  1012,   102, 20410,  1996,  2291,  1005,
         1055,  3754,  2000,  2709,  2000, 12398, 12113,  2044, 28667, 11475,
        10024,  3508,  1012,   102,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,   

In [9]:
#print(relevant_tokenized)
print(tokenized_data)

[{'requirement': 'The system shall support automatic calibration of all four cameras during initialization.', 'test_case': 'Validate that all four cameras are automatically calibrated within 2 seconds after power-on.', 'input_ids': tensor([  101,  1996,  2291,  4618,  2490,  6882, 10250, 12322,  8156,  1997,
         2035,  2176,  8629,  2076,  3988,  3989,  1012,   102,  9398,  3686,
         2008,  2035,  2176,  8629,  2024,  8073, 10250, 12322,  9250,  2306,
         1016,  3823,  2044,  2373,  1011,  2006,  1012,   102,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,   

In [14]:
input_ids = torch.stack([td['input_ids'] for td in tokenized_data])
attention_masks = torch.stack([td['attention_mask'] for td in tokenized_data])
labels = torch.tensor([td['label'] for td in tokenized_data])

print(input_ids)
print(attention_masks)
print(labels)

tensor([[ 101, 1996, 2291,  ...,    0,    0,    0],
        [ 101, 1996, 2291,  ...,    0,    0,    0],
        [ 101, 1996, 2291,  ...,    0,    0,    0],
        ...,
        [ 101, 1996, 8629,  ...,    0,    0,    0],
        [ 101, 1996, 8629,  ...,    0,    0,    0],
        [ 101, 1996, 8629,  ...,    0,    0,    0]])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])


In [15]:
from torch.utils.data import DataLoader, TensorDataset
train_data = TensorDataset(input_ids, attention_masks, labels)
train_dataloader = DataLoader(train_data, batch_size=16, shuffle=True)
#print(train_data)

In [114]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',  output_hidden_states=True, num_labels=2)
optimizer = AdamW(model.parameters(), lr=2e-5)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [115]:
epochs = 20
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        batch_input_ids, batch_attention_mask, batch_labels = tuple(t.to(device) for t in batch)

        optimizer.zero_grad()
        outputs = model(batch_input_ids, attention_mask=batch_attention_mask, labels=batch_labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_dataloader)}")

# Save the trained model
model.save_pretrained("bert_model_trained")
tokenizer.save_pretrained("bert_model_trained")
print("Model saved successfully!")

Epoch 1, Loss: 0.6960010826587677
Epoch 2, Loss: 0.6664261619249979
Epoch 3, Loss: 0.6418463687102
Epoch 4, Loss: 0.6045479079087576
Epoch 5, Loss: 0.48863115906715393
Epoch 6, Loss: 0.3757607887188594
Epoch 7, Loss: 0.3214042882124583
Epoch 8, Loss: 0.3343650350968043
Epoch 9, Loss: 0.2848556836446126
Epoch 10, Loss: 0.18124127139647803
Epoch 11, Loss: 0.14616287996371588
Epoch 12, Loss: 0.09563297778367996
Epoch 13, Loss: 0.08305407936374347
Epoch 14, Loss: 0.057539942053457103
Epoch 15, Loss: 0.07180805938939254
Epoch 16, Loss: 0.06527306822439034
Epoch 17, Loss: 0.05288117782523235
Epoch 18, Loss: 0.04000575436900059
Epoch 19, Loss: 0.03573157902186116
Epoch 20, Loss: 0.060329030733555555
Model saved successfully!


In [116]:
from transformers import BertTokenizer, BertModel
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
# Load the trained model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()  # Set the model to evaluation mode

new_requirements = [
    "Test the camera's ability to handle low-light conditions.",
    "Verify calibration accuracy after temperature changes.",
    "it is raining today.",
    "sachin scored a century at Perth."
]

# List of all test cases (from training data)
# Replace this with your actual list of test case descriptions
test_cases = df_expanded['Test Case Description'].tolist()

# Function to get BERT embeddings
def get_bert_embedding(text, prompt="For the new requirement give me the Best matching Test case to test my requirement"):  #"Given the requirement, match the best test case: "):
    input_text = prompt + text
    # Tokenize the text
    encoding = tokenizer.encode_plus(
        input_text,  # Input text
        max_length=128,
        truncation=True,
        padding='max_length',
        return_tensors='pt'
    )
    input_ids = encoding['input_ids']
    attention_mask = encoding['attention_mask']

    # Get the model outputs
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    # Extract the embeddings (we use the [CLS] token embedding)
    embeddings = outputs.last_hidden_state[:, 0, :]  # The [CLS] token embedding
    return embeddings.squeeze().numpy()

# Tokenize and get embeddings for new requirements
new_requirements_embeddings = [get_bert_embedding(req) for req in new_requirements]

# Tokenize and get embeddings for test cases
test_cases_embeddings = [get_bert_embedding(test_case) for test_case in test_cases]
#print(test_cases_embeddings) #
print(new_requirements_embeddings)



[array([-3.80989999e-01, -1.43141717e-01,  2.23043337e-01, -1.31337166e-01,
       -1.31051749e-01, -4.42171484e-01,  9.48304981e-02,  1.84621736e-01,
        3.70885104e-01, -5.53954959e-01,  4.86335997e-03,  1.91384211e-01,
        1.07472472e-01,  2.78715342e-01,  2.91523814e-01,  2.52299365e-02,
        8.17986727e-02,  4.85810488e-01,  2.71985829e-01, -7.12288097e-02,
       -4.06900086e-02, -7.09314048e-01,  2.51241326e-01,  2.58080304e-01,
       -1.05894849e-01, -1.83686092e-01, -1.77489400e-01,  3.21584940e-01,
       -6.01544641e-02, -1.30877182e-01, -7.34136999e-02,  5.00786483e-01,
       -5.44957757e-01, -1.62123129e-01,  1.56282902e-01, -1.24434911e-01,
        5.43486834e-01,  8.67255256e-02,  4.07152593e-01,  4.33336914e-01,
       -3.50161403e-01, -1.29449144e-01,  3.61838549e-01,  1.74579039e-01,
       -5.64131849e-02, -1.79489344e-01, -3.28086877e+00, -1.31569654e-01,
       -2.72711158e-01, -4.08714153e-02,  4.19810086e-01, -2.28534356e-01,
       -1.31563410e-01, 

In [117]:
# Now, for each new requirement, compare with all test cases and predict
best_matches = []
for req, req_embedding in zip(new_requirements, new_requirements_embeddings):
    # Calculate cosine similarity between the requirement embedding and all test case embeddings
    similarities = cosine_similarity([req_embedding], test_cases_embeddings)[0]  # Cosine similarity scores

    # Get the indices of the top 3 most similar test cases
    top_indices = similarities.argsort()[-3:][::-1]

    # Collect the best matching test cases
    best_matches_for_req = []
    for idx in top_indices:
        best_matches_for_req.append({
            'test_case': test_cases[idx],
            'similarity': similarities[idx]
        })

    best_matches.append({
        'requirement': req,
        'best_matches': best_matches_for_req
    })

# Display the best matching test cases for each new requirement
for match in best_matches:
    print(f"Requirement: {match['requirement']}")
    print("Top  best matching test cases:")
    for i, test_case in enumerate(match['best_matches']):
        print(f"{i+1}. Test Case: {test_case['test_case']}, Similarity: {test_case['similarity']:.4f}")
    print("\n")

# print(req_embedding)

Requirement: Test the camera's ability to handle low-light conditions.
Top  best matching test cases:
1. Test Case: Test system behavior for intermittent camera connections., Similarity: 0.9738
2. Test Case: Simulate camera disconnection and verify system behavior during calibration., Similarity: 0.9677
3. Test Case: Measure image contrast in bright and low-light conditions., Similarity: 0.9635


Requirement: Verify calibration accuracy after temperature changes.
Top  best matching test cases:
1. Test Case: Verify calibration accuracy at -40°C with simulated misalignments., Similarity: 0.9759
2. Test Case: Verify image quality under vibration., Similarity: 0.9736
3. Test Case: Delete calibration data and verify DTC logging., Similarity: 0.9710


Requirement: it is raining today.
Top  best matching test cases:
1. Test Case: Test latency after a sudden power cycle., Similarity: 0.9133
2. Test Case: Verify the system recalibrates successfully after a power cycle., Similarity: 0.9108
3. Te